In [47]:
import sys
reload(sys)
sys.setdefaultencoding("utf-8")
import numpy as np
import pandas
from pandas import DataFrame as df
from pandas import read_csv
import re
import sklearn
from sklearn.metrics import roc_auc_score
from sklearn.feature_extraction.text import CountVectorizer 
import itertools
from sklearn.cross_validation import cross_val_score
from sklearn.linear_model import SGDClassifier, LogisticRegression

In [48]:
train = pandas.read_csv("linear_train.txt", names=['Word', 'Label'])

In [49]:
train['Word'] = train['Word'].apply(lambda word: word.decode('utf-8'))
train['Length'] = train['Word'].apply(lambda word: len(word))
train['is_lower'] = train['Word'].apply(lambda word: int( word[0] >= 'а'.decode('utf-8')))
train['Word'] = train['Word'].apply(lambda x : x + "#")

In [50]:
train.head()

,Word,Label,Length,is_lower
0,Аалтонен#,1,8,0
1,Аар#,0,3,0
2,Аарон#,0,5,0
3,ААРОН#,0,5,0
4,Аарона#,0,6,0


In [51]:
test = pandas.read_csv('linear_test.txt', names=['Word'])
test['Word'] = test['Word'].apply(lambda word: word.decode('utf-8'))
test['Length'] = test['Word'].apply(lambda word: len(word))
test['is_lower'] = test['Word'].apply(lambda word: int( word[0] >= 'а'.decode('utf-8')))
test['Word'] = test['Word'].apply(lambda x : x + "#")

In [52]:
test.head()

,Word,Length,is_lower
0,﻿Аалто#,6,1
1,ААР#,3,0
2,Аара#,4,0
3,Ааре#,4,0
4,Аарон#,5,0


In [53]:
from scipy.sparse import hstack as hst
tr = CountVectorizer(ngram_range=(2, 8), analyzer='char_wb', binary=True)

In [ ]:
X_extra = train[['is_lower', 'Length']].values
X_train_features = tr.fit_transform(train['Word'])
X_train_features = hst([X_train_features, X_extra])

In [ ]:
X_test_extra = test[['is_lower','Length']].values
X_test_features = tr.transform(test['Word'])
X_test_features = hst([X_test_features, X_test_extra])

In [ ]:
algorithm = LogisticRegression().fit(X_train_features, train['Label'])

In [ ]:
pred = algorithm.predict_proba(X_test_features)
pandas.DataFrame(data=pred[:,1], columns=['Answer']).to_csv('submis.cvs', index=True, index_label='Id')